In [1]:
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

import pygame
import numpy as np

import torch
from IPython.display import clear_output
from IPython import display
import random
import matplotlib.pylab as plt

#ray.init(num_cpus=4, num_gpus=0)
ray.init(local_mode=True, num_gpus=0)


pygame 2.1.2 (SDL 2.0.16, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


2022-08-04 11:53:18,959	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265
2022-08-04 11:53:19,002	ERROR resource_spec.py:194 -- Could not parse gpu information.
Traceback (most recent call last):
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/_private/resource_spec.py", line 188, in resolve
    gpu_types = _get_gpu_types_gputil()
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/_private/resource_spec.py", line 301, in _get_gpu_types_gputil
    gpu_list = GPUtil.getGPUs()
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/GPUtil/GPUtil.py", line 102, in getGPUs
    deviceIds = int(vals[i])
ValueError: invalid literal for int() with base 10: 'Failed to initialize NVML: Driver/library version mismatch'


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.12', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '10.13.62.8', 'raylet_ip_address': '10.13.62.8', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-04_11-53-16_940088_141815/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-04_11-53-16_940088_141815/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-08-04_11-53-16_940088_141815', 'metrics_export_port': 51288, 'gcs_address': '10.13.62.8:53785', 'address': '10.13.62.8:53785', 'node_id': 'c898da70375e70fc01e1d0e814385dc292432f2f0712ae78ea78fe2f'})

In [2]:
config = {
#    "framework":'tf2',
#        "framework": "torch",
        "env":"'LunarLanderContinuous-v2'",  
#        "env_config":{'size':6, 'numHoles': 10},
        "num_workers": 10,
          # "model": {
          #    "custom_model": MyTorchModel,  # for torch users: "custom_model": MyTorchModel
          #    "custom_model_config": {},
          # },
        # 'num_envs_per_worker': 20,
        "create_env_on_driver": True,
#        "train_batch_size": 400,
         "n_step": 2, # multistep
        # "num_atoms": 4,  # for distributional DQN
        # "v_min": 0.0, 
        # "v_max": 1.0    
 #    "num_gpus": 1,
}


In [3]:
from ray import tune
tune_config = config.copy()
#tune_config["lr"] = tune.grid_search([0.0001])  # <- 0.5? again: ouch!

stop = {
    "training_iteration": 10,
    "episode_reward_mean": 195,
}

analysis =  tune.run(
    "PG",
    config=tune_config,
    stop=stop,
    checkpoint_at_end=True,  
    checkpoint_freq=5,  
    local_dir="checkPoints/",
    verbose=1,
    #resume=True,
   
)

2022-08-04 11:53:34,282	WARNING worker.py:525 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
:task_name:bundle_reservation_check_func
:actor_name:PGTrainer
[2022-08-04 11:53:34,335 E 141815 141815] core_worker.cc:1382: Pushed Error with JobID: 01000000 of type: task with message: ray::PGTrainer.__init__() (pid=141815, ip=10.13.62.8, repr=PGTrainer)
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 870, in __init__
    super().__init__(
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/tune/trainable.py", line 156, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 883, in setup
    self.config = self.merge_trainer_configs(
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/p

:task_name:bundle_reservation_check_func
:actor_name:PGTrainer


RayTaskError(AttributeError): [36mray::PGTrainer.get_auto_filled_metrics()::Exiting[39m (pid=141815, ip=10.13.62.8, repr=PGTrainer)
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/tune/trainable.py", line 263, in get_auto_filled_metrics
    "warmup_time": self._warmup_time,
AttributeError: 'PGTrainer' object has no attribute '_warmup_time'

2022-08-04 11:53:39,596	ERROR worker.py:94 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::PGTrainer.train()::Exiting (pid=141815, ip=10.13.62.8, repr=PGTrainer)
  File "/home/ajit/anaconda3/envs/rllib-stable/lib/python3.9/site-packages/ray/tune/trainable.py", line 357, in train
    if self._warmup_time is None:
AttributeError: 'PGTrainer' object has no attribute '_warmup_time'


In [ ]:
# The previous tune.run (the one we did before the exercise) returned an Analysis object, from which we can access any checkpoint
# (given we set checkpoint_freq or checkpoint_at_end to reasonable values) like so:
print(analysis)
# Get all trials (we only have one).
trials = analysis.trials
# Assuming, the first trial was the best, we'd like to extract this trial's best checkpoint "":
best_checkpoint = analysis.get_best_checkpoint(trial=trials[0], metric="episode_reward_mean", mode="max")
print(f"Found best checkpoint for trial #2: {best_checkpoint}")

In [ ]:
rllib_config = tune_config.copy()
rllib_config["lr"] = 0.0001
#rllib_config["train_batch_size"] = 4159
rllib_config["explore"] = False

# Restore a RLlib Trainer from the checkpoint.
new_trainer = DQNTrainer(config=rllib_config)
best_checkpoint = "/home/ajit/Desktop/myRLcases/misc/checkPoints/DQN/DQN_CartPole-v1_00fdc_00000_0_2022-07-26_13-51-52/checkpoint_000043/checkpoint-43"
new_trainer.restore(best_checkpoint)

In [ ]:
new_trainer.evaluate()

In [ ]:
env = gym.make('CartPole-v1')
obs = env.reset()
done = False
img = plt.imshow(env.render(mode='rgb_array')) 
while not done:
    obs, _, done, _ = env.step(new_trainer.compute_single_action(obs,explore=False))
    img.set_data(env.render(mode='rgb_array')) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
env.close()

In [1]:
import argparse
import pprint
from ray import tune
import ray

In [2]:
from ray.rllib.agents.pg.pg import (DEFAULT_CONFIG, PGTrainer as trainer)

In [ ]:
config = DEFAULT_CONFIG.copy()
config_update = {
    "env": 'LunarLanderContinuous-v2',
    "num_gpus": 0,
    "num_workers": 10,
    "evaluation_num_workers": 2,
    "evaluation_interval": 1,
    "create_env_on_driver": True,
    'num_envs_per_worker': 20,
}
config.update(config_update)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config)

In [4]:
ray.init()
tune.run(
    trainer,
    stop={"timesteps_total":20000000},
    config=config,
    checkpoint_freq=50,  
    local_dir="checkPoints/",
    verbose=1,
)

2022-07-28 14:24:38,020	INFO tune.py:747 -- Total run time: 1013.26 seconds (1012.57 seconds for the tuning loop).
